In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv('../content/drive/MyDrive/london/train.csv', header=None, index_col=None)
test = pd.read_csv('../content/drive/MyDrive/london/test.csv', header=None, index_col=None)
trainlabels = pd.read_csv('../content/drive/MyDrive/london/trainLabels.csv', header=None, index_col=None)

train.head()

In [ ]:
# Converting data into NumPy arrays
train = np.asarray(train)
trainlabels = np.asarray(trainlabels)
test = np.asarray(test)
trainlabels = trainlabels.ravel() # 1차원 배열

In [ ]:
print('train Shape:', train.shape, ',trainlabels Shape:', trainlabels.shape, ',test Shape:', test.shape)

gradient boosting classifier 불러오기

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

- train, test, train label 데이터가 있기 때문에 split하지 않음
- train과 train label 데이터로 훈련시켜 모델을 생성
- 후에 생성 모델로 predict한 값과 test 데이터 값을 이용해 예측 성능을 확인

In [ ]:
gb_null = GradientBoostingClassifier() # gbm 모델 초기화
gb_null.fit(train, trainlabels) # 모델 학습
grid_search_gb_null = GridSearchCV(gb_null, param_grid={}, scoring='accuracy', cv=10).fit(train, trainlabels) # train label이 없기 때문에 cross validation을 이용해 정확도 향상
gb_best_null = grid_search_gb_null.best_estimator_
print('Best estimator gbm_null:', grid_search_gb_null.best_estimator_)
print('Best parameters gmb_null:', grid_search_gb_null.best_params_)

파라미터 설정을 하지 않았기 때문에 best parameter가 나타나지 않음을 알 수 있음.

In [ ]:
gb_best_null.fit(train, trainlabels) # best 모델 학습
print(gb_best_null.predict(test)[:10]) # best 모델 예측 결과 10개 출력
print(gb_best_null.predict(test))

## GridSearchCV로 탐색한 best 하이퍼파라미터 조합의 prediction
pred = gb_best_null.predict(test)
print('Score for gbm:', cross_val_score(gb_best_null, train, trainlabels, cv=10, scoring='accuracy').mean()) #교차 검증을 통한 성능평가
print('best estimator gbm:', grid_search_gb_null.best_estimator_, 'Best Score', grid_search_gb_null.best_estimator_.score(train, trainlabels)) #훈련 데이터에 대한 단일 성능 지표

### 하이퍼파라미터 튜닝

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Gradient Boosting Classifier 모델 초기화
gb = GradientBoostingClassifier()

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 조합 탐색을 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 조합 탐색
grid_search_gb = GridSearchCV(gb, param_grid=param_grid, scoring='accuracy', cv=10)
grid_search_gb.fit(train, trainlabels)

# 최적의 파라미터 출력
print('Best parameters:', grid_search_gb.best_params_)

n_estimator이 클수록, learning rate는 높을수록, max_depth가 깊을수록 성능이 좋은 모델이 도출됨을 알 수 있음.
하이퍼파라미터 탐색을 하니 결과 도출까지 시간이 오래 걸림.(이는 gbm의 단점이기도 함)

In [ ]:
gb_best = grid_search_gb.best_estimator_

In [ ]:
gb_best.fit(train, trainlabels) # best 모델 학습
print(gb_best.predict(test)[:10]) # best 모델 예측 결과 10개 출력

## GridSearchCV로 탐색한 best 하이퍼파라미터 조합의 prediction
pred1 = gb_best.predict(test)
print('Score for gbm:', cross_val_score(gb_best, train, trainlabels, cv=10, scoring='accuracy').mean()) #교차 검증을 통한 성능평가
print('best estimator gbm:', grid_search_gb.best_estimator_, 'Best Score', grid_search_gb.best_estimator_.score(train, trainlabels)) #훈련 데이터에 대한 단일 성능 지표

In [ ]:
print(gb_best.predict(test)) #best model을 통해 예측한 testlabel로 캐글에 올린 결과 accuracy 0.885 나옴

하이퍼파라미터 조정을 한 후 모델을 학습시킨 결과 성능이 향상됨.
best score이 0.985에서 1로 향상.

In [ ]:
submit_dict = {'Id': np.arange(1, len(pred1) + 1), 'Solution': pd.Series(pred1)}
my_submission = pd.DataFrame(submit_dict)
my_submission.to_csv('submission.csv', index=False)

In [ ]:
from google.colab import files
files.download('submission.csv')

### 다른 하이퍼파라미터 추가 후 모델 학습
(하이퍼파라미터 종류가 많아 학습 시간이 오래걸림)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Gradient Boosting Classifier 모델 초기화
gb_1 = GradientBoostingClassifier()

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 조합 탐색을 위한 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.6, 0.8, 1.0]
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 조합 탐색
grid_search_gb_1 = GridSearchCV(gb_1, param_grid=param_grid, scoring='accuracy', cv=10)
grid_search_gb_1.fit(train, trainlabels)

# 최적의 파라미터 출력
print('Best parameters:', grid_search_gb_1.best_params_)

In [ ]:
gb_1_best = grid_search_gb_1.best_estimator_

In [ ]:
gb_1_best.fit(train, trainlabels) # best 모델 학습
print(gb_1_best.predict(test)[:10]) # best 모델 예측 결과 10개 출력

## GridSearchCV로 탐색한 best 하이퍼파라미터 조합의 prediction
pred = gb_1_best.predict(test)
print('Score for gbm:', cross_val_score(gb_1_best, train, trainlabels, cv=10, scoring='accuracy').mean()) #교차 검증을 통한 성능평가
print('best estimator gbm:', grid_search_gb_1.best_estimator_, 'Best Score', grid_search_gb_1.best_estimator_.score(train, trainlabels)) #훈련 데이터에 대한 단일 성능 지표